### Script to plot maps using raw WRF output

In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib
from matplotlib import ticker, cm, colors
import matplotlib.pyplot as plt
import sys
import cartopy
import subprocess

#### Main Settings

In [ ]:
# Model file to plot
it=0
# Model level to plot
ikplot = 30

# #### Directories

storm='haiyan'
# storm='maria'

main = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/tc_ens/"
memb='memb_01'
test='ctl'
# datdir2 = 'post/d02/'
datdir = main+storm+'/'+memb+'/'+test+'/'#+datdir2
# figdir = "/home/jamesrup/figures/tc/ens/"+storm+'/'

In [ ]:
##### Get dimensions
process = subprocess.Popen(['ls '+datdir+'wrfout_d02_*'],shell=True,
    stdout=subprocess.PIPE,universal_newlines=True)
output = process.stdout.readlines()
nt = len(output)
wrffil = output[0].strip() #[3]
varfil_main = Dataset(wrffil)
lat = varfil_main.variables['XLAT'][:][0] # deg
lon = varfil_main.variables['XLONG'][:][0] # deg
lat1d = lat[:,0]
lon1d = lon[0,:]
nx1 = lat1d.size
nx2 = lon1d.size
nz = varfil_main.dimensions['bottom_top'].size
pres = varfil_main.variables['PB'][0,:,0,0]*1e-2 # Pa --> hPa
varfil_main.close()

In [ ]:
def var_read(file,varname,ikread):
    varfil_main = Dataset(file)
    var = varfil_main.variables[varname][0,ikread,:,:]
    varfil_main.close()
    return var
def var_read_2d(file,varname):
    varfil_main = Dataset(file)
    var = varfil_main.variables[varname][0,:,:]
    varfil_main.close()
    return var

In [ ]:
# Function to account for crossing of the Intl Date Line
def dateline_lon_shift(lon_in, reverse):
    if reverse == 0:
        lon_offset = np.zeros(lon_in.shape)
        lon_offset[np.where(lon_in < 0)] += 360
    else:
        lon_offset = np.zeros(lon_in.shape)
        lon_offset[np.where(lon_in > 180)] -= 360
    # return lon_in + lon_offset
    return lon_offset

# Check for crossing Date Line
if (lon.min() < 0) and (lon.max() > 0):
    offset = 180
    lon_offset = dateline_lon_shift(lon, reverse=0)
else:
    offset = 0
    lon_offset = 0
    clon_offset = 0

lon_offset_plt = lon + lon_offset
lon_offset_plt -= offset

---
### Plotting routines

In [ ]:
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

plt_area=[lon1d[0], lon1d[-1], lat1d[0], lat1d[-1]] # W,E,S,N

##### Plotting functions

In [ ]:
# wind barbs
def plot_wind(ax, u, v, lon, lat, skip, length):
    spacing=skip #barbspacing (smaller if zoomed in)
    mps_to_kts=1.94384 # conversion factor from m/s to knots for barbs
    uplt = u * mps_to_kts
    vplt = v * mps_to_kts
    ax.barbs(lon[::spacing,::spacing], lat[::spacing,::spacing], 
             uplt[::spacing,::spacing], vplt[::spacing,::spacing], 
             zorder=2, length=length, color='gray')

In [ ]:
def run_plot(plot_name, it, ikread, offset, pres, lon_offset_plt):

    # Switches (default settings)
    i2d=False   # switch on if the data read-in needs to be done in 2D
    dosym=True  # switch off to specify min colorbar setting
    dolog=False # switch for logarithmic color scale

    if plot_name == 'vert_motion':
        # Vertical motion
        vartag='W'
        scale=1e2 # m/s --> cm/s
        unittag='cm/s'
        cmax=50
        cmap='RdGy_r'
    elif plot_name == 'height':
        # Perturbation geopotential height (from initial state)
        vartag='PH'
        scale=1./9.81 # divide off gravity to get height; m2/s2 --> m
        unittag='m'
        cmax=20
        cmap='RdGy_r'
    elif plot_name == 'uwind' or plot_name == 'vwind':
        # Zonal/meridional wind
        if plot_name == 'uwind': vartag='U'
        if plot_name == 'vwind': vartag='V'
        scale=1.
        unittag='m/s'
        cmax=10
        cmap='BrBG_r'
    elif plot_name == 'wspd':
        # Wind speed
        vartag='U'
        scale=1.
        unittag='m/s'
        cmax=20
        cmap='cubehelix'
    elif plot_name == 'olr':
        # OLR
        i2d=True
        vartag='LWUPT'
        scale=1.
        unittag='W/m^2'
        cmin=50
        cmax=320
        dosym=False
        cmap='RdGy'
    elif plot_name == 'qrain':
        # QRAIN - rain water mixing ratio
        vartag='QRAIN'
        scale=1. # divide off gravity to get height; m2/s2 --> m
        unittag='kg/kg'
        cmin=1e-6
        cmax=1e-4
        dolog=True
        cmap='RdGy_r'
        cmap='gist_gray'

    ## Read variables
    file = output[it].strip()

    hr_tag = str(it)
    title = vartag+', time step='+hr_tag+',  k-level='+str(ikread)+' (p = '+str(int(pres[ikread]))+' hPa)'

    # Read var
    if i2d:
        pltvar = var_read_2d(file,vartag) * scale
    else:
        pltvar = var_read(file,vartag,ikread) * scale

    # Modifications
    if plot_name == 'pres' or plot_name == 'height':
        pltvar -= var_read(output[0].strip(),vartag,ikread) * scale
    elif plot_name == 'uwind':
        pltvar = 0.5*(pltvar[:,0:nx2] + pltvar[:,1:nx2+1])
    elif plot_name == 'vwind':
        pltvar = 0.5*(pltvar[0:nx1,:] + pltvar[1:nx1+1,:])
    elif plot_name == 'wspd':
        ui = 0.5*(pltvar[:,0:nx2] + pltvar[:,1:nx2+1])
        v = var_read(output[0].strip(),'V',ikread)
        vi = 0.5*(v[0:nx1,:] + v[1:nx1+1,:])
        pltvar = np.sqrt( ui**2 + vi**2)
    elif plot_name == 'qrain':
        # pltvar = np.ma.masked_where(pltvar == cmin, pltvar)
        # pltvar = np.ma.filled(pltvar, fill_value=cmin)
        loc=np.where(pltvar <= cmin)
        pltvar[loc]=cmin

    # Color scale
    nlevs=71
    if dosym:
        delta=2*cmax/nlevs
        clevs = np.arange(-1*cmax,cmax+delta,delta)
    else:
        delta=(cmax-cmin)/nlevs
        clevs = np.arange(cmin,cmax+delta,delta)

    # create figure
    # do_plot(pltvar, title, clevs, cmap, lon_offset_plt, lat, unittag, plt_area)

    fig = plt.figure(figsize=(20,10))
    proj = cartopy.crs.PlateCarree(central_longitude=offset)
    ax = fig.add_subplot(111,projection=proj)
    ax.set_title(title, fontsize=20)

    # fill contour
    if dolog:
        im = ax.contourf(lon_offset_plt, lat, pltvar, cmap=cmap, alpha=0.9,
                            extend='both', zorder=2, norm=colors.LogNorm(vmin=cmin, vmax=cmax))
        ticks=ticker.LogLocator()
    else:
        im = ax.contourf(lon_offset_plt, lat, pltvar, clevs, cmap=cmap, alpha=0.9,
                            extend='both', zorder=2)
        ticks=ticker.AutoLocator()

    cbar = plt.colorbar(im, ax=ax, shrink=0.65, ticks=ticks)
    cbar.ax.set_ylabel(unittag)

    # add map features
    ax.add_feature(cartopy.feature.LAND,facecolor="lightgray") #land color
    # ax.add_feature(cartopy.feature.OCEAN) #ocean color
    ax.add_feature(cartopy.feature.COASTLINE)
    # ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
    ax.set_extent(plt_area)

    plt.show()
    plt.close()

##### Main plotting code

In [ ]:

ikread=25 # Set vertical level

plot_name='vert_motion'
for it in range(0,6):
    run_plot(plot_name, it, ikread, offset, pres, lon_offset_plt)

plot_name='height'
for it in range(5,6):
    run_plot(plot_name, it, ikread, offset, pres, lon_offset_plt)

In [ ]:

ikread=25 # Set vertical level

for it in range(96,97):

    plot_name='vert_motion'
    run_plot(plot_name, it, ikread, offset, pres, lon_offset_plt)

    plot_name='height'
    run_plot(plot_name, it, ikread, offset, pres, lon_offset_plt)

    ikread=6 # Set vertical level
    plot_name='qrain'
    run_plot(plot_name, it, ikread, offset, pres, lon_offset_plt)

    ikread=25 # Set vertical level
    plot_name='olr'
    run_plot(plot_name, it, ikread, offset, pres, lon_offset_plt)

#     plot_name='uwind'
#     run_plot(plot_name, it, ikread, offset, pres, lon_offset_plt)